In [1]:
from methods import *
import os, shutil

In [2]:
# We have percent_data/percentages/dataset/train_orig.txt

#load hyperparameters
sizes = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

folders_list =  ['percent_data/' + str(int(size*100)) for size in sizes]
#size_folders = ['size_data_f1/' + size for size in sizes]

#datasets
datasets = ['sst2']

#number of output classes
num_classes_list = [2, 2, 2, 6, 2]

#number of augmentations per original sentence
n_aug_list_dict = {'size_data_f1/1_tiny': [32, 32, 32, 32, 32], 
					'size_data_f1/2_small': [32, 32, 32, 32, 32],
					'size_data_f1/3_standard': [16, 16, 16, 16, 4],
					'size_data_f1/4_full': [16, 16, 16, 16, 4]}

if not os.path.isdir('size_data_f1'):
    os.mkdir('size_data_f1')
#number of words for input
input_size_list = [50, 50, 40, 25, 25]


# Number of words for input
input_size_list = [50,50,40,25,25]

#word2vec dictionary
huge_word2vec = 'word2vec/glove.840B.300d.txt'
word2vec_len = 300

In [3]:
def run_cnn(train_file, test_file, num_classes, percent_dataset):

    #initialize model
    model = build_cnn(input_size, word2vec_len, num_classes)

    #load data
    train_x, train_y = get_x_y(train_file, num_classes, word2vec_len, input_size, word2vec, percent_dataset)
    test_x, test_y = get_x_y(test_file, num_classes, word2vec_len, input_size, word2vec, 1)

    #implement early stopping
    callbacks = [EarlyStopping(monitor='val_loss', patience=3)]

    #train model
    model.fit(	train_x, 
                train_y, 
                epochs=20, 
                callbacks=callbacks,
                validation_split=0.1, 
                batch_size=1024, 
                shuffle=True, 
                verbose=1)
    #model.save('checkpoints/lol')
    #model = load_model('checkpoints/lol')

    #evaluate model
    y_pred = model.predict(test_x)
    test_y_cat = one_hot_to_categorical(test_y)
    y_pred_cat = one_hot_to_categorical(y_pred)
    acc = accuracy_score(test_y_cat, y_pred_cat)

    #clean memory???
    train_x, train_y, test_x, test_y, model = None, None, None, None, None
    gc.collect()

    #return the accuracy
    #print("data with shape:", train_x.shape, train_y.shape, 'train=', train_file, 'test=', test_file, 'with fraction', percent_dataset, 'had acc', acc)
    return acc

In [8]:
#for each method
a_method = 'weda' #### TOGGLE EDA/WEDA
tfidf_func = 2

writer_cnn = open('outputs_percent/cnn_' + a_method + '_' +'tfidf'+str(tfidf_func)+'_'+ get_now_str() + '.txt', 'w')

#for storing the performances
performances_cnn = {size_folder:[] for size_folder in folders_list}

#for each percentage dataset
for size_folder in folders_list:

    writer_cnn.write(size_folder + '\n')

    #get all six datasets
    dataset_folders = [size_folder + '/' + s for s in datasets]



    #for each dataset
    for i in range(len(dataset_folders)):

        #initialize all the variables
        dataset_folder = dataset_folders[i]
        dataset = datasets[i]
        num_classes = num_classes_list[i]
        input_size = input_size_list[i]
        word2vec_pickle = dataset + '/word2vec.p'
        word2vec = load_pickle(word2vec_pickle)

        

        train_path = dataset_folder + '/train_weda_pct_tfidf'+str(tfidf_func)+'.txt' #### TOGGLE EDA/WEDA
        test_path = dataset + '/test.txt'
        acc = run_cnn(train_path, test_path, num_classes, percent_dataset=1)
        print("cnn aug ", acc, dataset_folder)

        performances_cnn[size_folder].append(acc)
        """
        acc = run_rnn(train_path, test_path, num_classes, percent_dataset=1)
        performances_rnn[alpha].append(acc)
        print("rnn aug", acc)
        """

writer_cnn.write(str(performances_cnn) + '\n')
#writer_rnn.write(str(performances_rnn)+"\n")
for size_folder in performances_cnn:
    line = str(size_folder) + ' : ' + str(sum(performances_cnn[size_folder])/len(performances_cnn[size_folder]))
    writer_cnn.write(line + '\n')
    print(line)
"""
for alpha in performances_rnn:
    line = str(alpha) + ' : ' + str(sum(performances_rnn[alpha])/len(performances_rnn[alpha]))
    writer_rnn.write(line + '\n')
    print(line)
"""
print(performances_cnn)

writer_cnn.close()    


Epoch 1/20
1/1 [==============================] - 1s 649ms/step - loss: 0.6929 - accuracy: 0.4932 - val_loss: 0.6724 - val_accuracy: 0.7260
Epoch 2/20
1/1 [==============================] - 0s 161ms/step - loss: 0.6664 - accuracy: 0.7960 - val_loss: 0.6558 - val_accuracy: 0.6575
Epoch 3/20
1/1 [==============================] - 0s 168ms/step - loss: 0.6394 - accuracy: 0.7732 - val_loss: 0.6306 - val_accuracy: 0.8082
Epoch 4/20
1/1 [==============================] - 0s 164ms/step - loss: 0.6090 - accuracy: 0.9376 - val_loss: 0.6013 - val_accuracy: 0.9589
Epoch 5/20
1/1 [==============================] - 0s 195ms/step - loss: 0.5785 - accuracy: 0.9802 - val_loss: 0.5729 - val_accuracy: 0.9726
Epoch 6/20
1/1 [==============================] - 0s 186ms/step - loss: 0.5478 - accuracy: 0.9802 - val_loss: 0.5454 - val_accuracy: 0.9726
Epoch 7/20
1/1 [==============================] - 0s 172ms/step - loss: 0.5158 - accuracy: 0.9833 - val_loss: 0.5182 - val_accuracy: 0.9726
Epoch 8/20
1/1 [====

7/7 [==============================] - 1s 146ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0377 - val_accuracy: 0.9919
Epoch 19/20
7/7 [==============================] - 1s 145ms/step - loss: 0.0026 - accuracy: 1.0000 - val_loss: 0.0368 - val_accuracy: 0.9919
Epoch 20/20
7/7 [==============================] - 1s 146ms/step - loss: 0.0023 - accuracy: 1.0000 - val_loss: 0.0362 - val_accuracy: 0.9919
cnn aug  0.8073184676958262 percent_data/10/sst2
Epoch 1/20
13/13 [==============================] - 3s 176ms/step - loss: 0.6719 - accuracy: 0.5987 - val_loss: 0.5815 - val_accuracy: 0.7842
Epoch 2/20
13/13 [==============================] - 2s 156ms/step - loss: 0.5394 - accuracy: 0.8075 - val_loss: 0.4404 - val_accuracy: 0.8166
Epoch 3/20
13/13 [==============================] - 2s 156ms/step - loss: 0.3903 - accuracy: 0.8444 - val_loss: 0.3444 - val_accuracy: 0.8606
Epoch 4/20
13/13 [==============================] - 2s 159ms/step - loss: 0.2774 - accuracy: 0.8950 - val_loss: 0.

26/26 [==============================] - 4s 158ms/step - loss: 8.9068e-04 - accuracy: 1.0000 - val_loss: 0.0464 - val_accuracy: 0.9865
Epoch 16/20
26/26 [==============================] - 4s 166ms/step - loss: 5.7983e-04 - accuracy: 1.0000 - val_loss: 0.0461 - val_accuracy: 0.9871
Epoch 17/20
26/26 [==============================] - 4s 172ms/step - loss: 4.0580e-04 - accuracy: 1.0000 - val_loss: 0.0463 - val_accuracy: 0.9871
Epoch 18/20
26/26 [==============================] - 4s 169ms/step - loss: 2.9267e-04 - accuracy: 1.0000 - val_loss: 0.0465 - val_accuracy: 0.9868
Epoch 19/20
26/26 [==============================] - 4s 166ms/step - loss: 2.2214e-04 - accuracy: 1.0000 - val_loss: 0.0467 - val_accuracy: 0.9868
cnn aug  0.8267581475128645 percent_data/40/sst2
Epoch 1/20
33/33 [==============================] - 7s 177ms/step - loss: 0.6057 - accuracy: 0.6782 - val_loss: 0.4139 - val_accuracy: 0.8006
Epoch 2/20
33/33 [==============================] - 5s 161ms/step - loss: 0.3631 - acc

46/46 [==============================] - 7s 153ms/step - loss: 9.9986e-04 - accuracy: 1.0000 - val_loss: 0.0605 - val_accuracy: 0.9786
Epoch 13/20
46/46 [==============================] - 7s 153ms/step - loss: 6.6985e-04 - accuracy: 1.0000 - val_loss: 0.0614 - val_accuracy: 0.9789
Epoch 14/20
46/46 [==============================] - 7s 153ms/step - loss: 4.6710e-04 - accuracy: 1.0000 - val_loss: 0.0626 - val_accuracy: 0.9787
cnn aug  0.8256146369353916 percent_data/70/sst2
Epoch 1/20
52/52 [==============================] - 10s 180ms/step - loss: 0.5823 - accuracy: 0.6989 - val_loss: 0.3799 - val_accuracy: 0.8360
Epoch 2/20
52/52 [==============================] - 9s 175ms/step - loss: 0.3266 - accuracy: 0.8662 - val_loss: 0.2568 - val_accuracy: 0.9055
Epoch 3/20
52/52 [==============================] - 8s 159ms/step - loss: 0.1847 - accuracy: 0.9451 - val_loss: 0.1504 - val_accuracy: 0.9533
Epoch 4/20
52/52 [==============================] - 8s 154ms/step - loss: 0.0792 - accuracy: 0.

In [4]:
### For training on the original data, without augmentations
#for each method
a_method = 'no_aug' #### TOGGLE EDA/WEDA

writer_cnn = open('outputs_percent/cnn_' + a_method + '_' + get_now_str() + '.txt', 'w')

#for storing the performances
performances_cnn = {size_folder:[] for size_folder in folders_list}

#for each percentage dataset
for size_folder in folders_list:

    writer_cnn.write(size_folder + '\n')

    #get all six datasets
    dataset_folders = [size_folder + '/' + s for s in datasets]



    #for each dataset
    for i in range(len(dataset_folders)):

        #initialize all the variables
        dataset_folder = dataset_folders[i]
        dataset = datasets[i]
        num_classes = num_classes_list[i]
        input_size = input_size_list[i]
        word2vec_pickle = dataset + '/word2vec.p'
        word2vec = load_pickle(word2vec_pickle)

        

        train_path = dataset_folder + '/train_orig.txt' #### TOGGLE EDA/WEDA
        test_path = dataset + '/test.txt'
        acc = run_cnn(train_path, test_path, num_classes, percent_dataset=1)
        print("cnn aug ", acc, dataset_folder)

        performances_cnn[size_folder].append(acc)
        """
        acc = run_rnn(train_path, test_path, num_classes, percent_dataset=1)
        performances_rnn[alpha].append(acc)
        print("rnn aug", acc)
        """

writer_cnn.write(str(performances_cnn) + '\n')
#writer_rnn.write(str(performances_rnn)+"\n")
for size_folder in performances_cnn:
    line = str(size_folder) + ' : ' + str(sum(performances_cnn[size_folder])/len(performances_cnn[size_folder]))
    writer_cnn.write(line + '\n')
    print(line)
"""
for alpha in performances_rnn:
    line = str(alpha) + ' : ' + str(sum(performances_rnn[alpha])/len(performances_rnn[alpha]))
    writer_rnn.write(line + '\n')
    print(line)
"""
print(performances_cnn)

writer_cnn.close()    


Epoch 1/20
1/1 [==============================] - 1s 861ms/step - loss: 0.6983 - accuracy: 0.4769 - val_loss: 0.6910 - val_accuracy: 0.6250
Epoch 2/20
1/1 [==============================] - 0s 55ms/step - loss: 0.6438 - accuracy: 0.9077 - val_loss: 0.6905 - val_accuracy: 0.6250
Epoch 3/20
1/1 [==============================] - 0s 56ms/step - loss: 0.5933 - accuracy: 0.9692 - val_loss: 0.6923 - val_accuracy: 0.5000
Epoch 4/20
1/1 [==============================] - 0s 57ms/step - loss: 0.5517 - accuracy: 0.9538 - val_loss: 0.6950 - val_accuracy: 0.5000
Epoch 5/20
1/1 [==============================] - 0s 58ms/step - loss: 0.5111 - accuracy: 0.9538 - val_loss: 0.6970 - val_accuracy: 0.5000
cnn aug  0.534019439679817 percent_data/1/sst2
Epoch 1/20
1/1 [==============================] - 1s 506ms/step - loss: 0.7147 - accuracy: 0.5000 - val_loss: 0.6831 - val_accuracy: 0.4865
Epoch 2/20
1/1 [==============================] - 0s 99ms/step - loss: 0.6743 - accuracy: 0.5181 - val_loss: 0.6777 -

2/2 [==============================] - 0s 110ms/step - loss: 0.3517 - accuracy: 0.8973 - val_loss: 0.4914 - val_accuracy: 0.7432
Epoch 14/20
2/2 [==============================] - 0s 106ms/step - loss: 0.3217 - accuracy: 0.9054 - val_loss: 0.4799 - val_accuracy: 0.7432
Epoch 15/20
2/2 [==============================] - 0s 115ms/step - loss: 0.2951 - accuracy: 0.9109 - val_loss: 0.4745 - val_accuracy: 0.7432
Epoch 16/20
2/2 [==============================] - 0s 118ms/step - loss: 0.2684 - accuracy: 0.9204 - val_loss: 0.4785 - val_accuracy: 0.7568
Epoch 17/20
2/2 [==============================] - 0s 115ms/step - loss: 0.2448 - accuracy: 0.9263 - val_loss: 0.4635 - val_accuracy: 0.7432
Epoch 18/20
2/2 [==============================] - 0s 121ms/step - loss: 0.2151 - accuracy: 0.9439 - val_loss: 0.4559 - val_accuracy: 0.7703
Epoch 19/20
2/2 [==============================] - 0s 109ms/step - loss: 0.1947 - accuracy: 0.9506 - val_loss: 0.4587 - val_accuracy: 0.7500
Epoch 20/20
2/2 [========

4/4 [==============================] - 1s 162ms/step - loss: 0.6205 - accuracy: 0.6329 - val_loss: 0.5987 - val_accuracy: 0.6802
Epoch 4/20
4/4 [==============================] - 1s 165ms/step - loss: 0.5789 - accuracy: 0.7315 - val_loss: 0.5500 - val_accuracy: 0.7680
Epoch 5/20
4/4 [==============================] - 1s 169ms/step - loss: 0.5325 - accuracy: 0.8042 - val_loss: 0.5096 - val_accuracy: 0.7883
Epoch 6/20
4/4 [==============================] - 1s 160ms/step - loss: 0.4855 - accuracy: 0.8007 - val_loss: 0.4621 - val_accuracy: 0.8243
Epoch 7/20
4/4 [==============================] - 1s 175ms/step - loss: 0.4371 - accuracy: 0.8286 - val_loss: 0.4274 - val_accuracy: 0.8176
Epoch 8/20
4/4 [==============================] - 1s 160ms/step - loss: 0.3937 - accuracy: 0.8460 - val_loss: 0.4086 - val_accuracy: 0.8288
Epoch 9/20
4/4 [==============================] - 1s 165ms/step - loss: 0.3572 - accuracy: 0.8619 - val_loss: 0.3935 - val_accuracy: 0.8243
Epoch 10/20
4/4 [==============

7/7 [==============================] - 1s 150ms/step - loss: 0.3125 - accuracy: 0.8744 - val_loss: 0.3445 - val_accuracy: 0.8351
Epoch 8/20
7/7 [==============================] - 1s 149ms/step - loss: 0.2823 - accuracy: 0.8927 - val_loss: 0.3401 - val_accuracy: 0.8378
Epoch 9/20
7/7 [==============================] - 1s 149ms/step - loss: 0.2477 - accuracy: 0.9186 - val_loss: 0.3586 - val_accuracy: 0.8311
Epoch 10/20
7/7 [==============================] - 1s 151ms/step - loss: 0.2210 - accuracy: 0.9285 - val_loss: 0.3455 - val_accuracy: 0.8324
Epoch 11/20
7/7 [==============================] - 1s 151ms/step - loss: 0.1874 - accuracy: 0.9494 - val_loss: 0.3368 - val_accuracy: 0.8432
Epoch 12/20
7/7 [==============================] - 1s 152ms/step - loss: 0.1551 - accuracy: 0.9652 - val_loss: 0.3398 - val_accuracy: 0.8405
Epoch 13/20
7/7 [==============================] - 1s 148ms/step - loss: 0.1293 - accuracy: 0.9757 - val_loss: 0.3486 - val_accuracy: 0.8419
Epoch 14/20
7/7 [==========